In [1]:
#Importing
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import seaborn as sns
import os
%matplotlib inline
plt.rcParams['figure.figsize'] = [12, 7]

In [2]:
#Reading in file
data = pd.read_csv('train-set.csv')  #Change the file name to your data
print("dimension of data: {}".format(data.shape))
data.head()

dimension of data: (8523, 12)


,ProductID,Weight,FatContent,ProductVisibility,ProductType,MRP,OutletID,EstablishmentYear,OutletSize,LocationType,OutletType,OutletSales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [3]:
#Preparing file
data.drop('ProductID', axis=1, inplace=True)
data.drop('Weight', axis=1, inplace=True)
data.drop('FatContent', axis=1, inplace=True)
data.drop('ProductVisibility', axis=1, inplace=True)
data.drop('ProductType', axis=1, inplace=True)
data.drop('OutletID', axis=1, inplace=True)
data.drop('EstablishmentYear', axis=1, inplace=True)
data.drop('OutletSize', axis=1, inplace=True)
data.drop('LocationType', axis=1, inplace=True)
data.drop('OutletType', axis=1, inplace=True)
le=LabelEncoder()
#data["Gender"] = le.fit_transform(data["Gender"])   # Change here, delete it or add more rows if necessary
data.head()

,MRP,OutletSales
0,249.8092,3735.1380
1,48.2692,443.4228
2,141.6180,2097.2700
3,182.0950,732.3800
4,53.8614,994.7052


In [4]:
pca = PCA(n_components = 2, random_state = 0) 
pca = pca.fit(data)
df_reduced = pca.transform(data)
df_reduced = pd.DataFrame(df_reduced, columns = ['Dimension 1', 'Dimension 2'])

In [5]:
df_reduced.head()

,Dimension 1,Dimension 2
0,1555.770690,76.586920
1,-1739.414636,-56.683481
2,-83.987906,2.366516
3,-1447.745746,71.124449
4,-1188.134755,-62.518732


/Users/ethan/opt/anaconda3/lib/python3.7/site-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning


In [6]:
sns.scatterplot(df_reduced['Dimension 1'], df_reduced['Dimension 2'], alpha=0.4)
plt.show()

In [7]:
wcss = []
for i in range(1, 11):
    km = KMeans(n_clusters = i, init = 'k-means++', max_iter = 300, n_init = 10)
    km.fit(df_reduced)
    wcss.append(km.inertia_)
plt.plot(range(1, 11), wcss)
plt.title('The Elbow Method')
plt.xlabel('No. of Clusters')
plt.ylabel('wcss')
plt.show()

In [8]:
for i in range(2, 15):
    km = KMeans(n_clusters = i)
    km.fit(df_reduced)
    label = km.labels_
    sil_coeff = silhouette_score(df_reduced, label, metric='euclidean')
    print("For n_clusters={}, The Silhouette Coefficient is {}".format(i, sil_coeff))

For n_clusters=2, The Silhouette Coefficient is 0.6149172579865921
For n_clusters=3, The Silhouette Coefficient is 0.5819300925110169
For n_clusters=4, The Silhouette Coefficient is 0.5563180058396607
For n_clusters=5, The Silhouette Coefficient is 0.5539626443357186
For n_clusters=6, The Silhouette Coefficient is 0.5427248540358317
For n_clusters=7, The Silhouette Coefficient is 0.5362244854661349
For n_clusters=8, The Silhouette Coefficient is 0.5259733466562021
For n_clusters=9, The Silhouette Coefficient is 0.5265127644867615
For n_clusters=10, The Silhouette Coefficient is 0.5148057537378193
For n_clusters=11, The Silhouette Coefficient is 0.5096205329034658
For n_clusters=12, The Silhouette Coefficient is 0.5069181504287175
For n_clusters=13, The Silhouette Coefficient is 0.5005731564291565
For n_clusters=14, The Silhouette Coefficient is 0.4987224802667072


In [9]:
km = KMeans(n_clusters=5)
clusters = km.fit_predict(df_reduced)
df_pred = pd.concat([df_reduced, pd.DataFrame(clusters, columns=['pred'])], axis=1)

In [10]:
sns.scatterplot(x=df_pred['Dimension 1'], y=df_pred['Dimension 2'],alpha=0.6, hue=df_pred['pred'], palette="deep")
plt.scatter(km.cluster_centers_[:,0], km.cluster_centers_[:,1], s=200 ,marker='s', c='red', alpha=0.3, label='Centroids')
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.title('Clustering using k-means', size=16)
plt.show()